## Información no Estructurada
# Práctica 3.2 &mdash; Búsqueda Web y Recomendación - Relevance Feedback
### Autores: Iñigo Gómez Carvajal y Jon Zorrilla Gamboa 

# 0.  Ejercicios

## Ejemplo diapositivas

Matriz de frecuencias término / documento

In [1]:
import numpy as np
import pandas as pd

F = np.array([[0, 1, 2, 2, 1, 1, 2, 2, 2, 1, 6, 4], # d1
              [0, 1, 4, 2, 2, 0, 0, 4, 9, 1, 10, 0], # d2
              [1, 0, 2, 0, 0, 0, 1, 2, 0, 1, 1, 0], # d3
              [1, 1, 0, 1, 9, 1, 28, 1, 7, 1, 56, 8], # d4
              [0, 0, 0, 1, 7, 2, 30, 1, 6, 1, 66, 13], # d5
              [0, 0, 1, 3, 3, 0, 1, 1, 1, 0, 5, 8], # d6
              [1, 0, 2, 2, 4, 0, 0, 0, 0, 0, 56, 1], # d7
              [0, 0, 1, 8, 1, 0, 5, 1, 0, 1, 19, 1], # d8
              [7, 0, 0, 0, 0, 0, 0, 0, 0, 1, 11, 0], # d9
              [1, 0, 0, 1, 4, 0, 0, 0, 0, 0, 22, 5], # d10
              [0, 0, 1, 6, 0, 0, 1, 0, 1, 0, 18, 1], # d11
              [0, 0, 0, 3, 0, 0, 1, 0, 0, 0, 16, 0], # d12
              [1, 1, 0, 2, 2, 0, 2, 0, 4, 0, 15, 0], # d13
              [1, 1, 1, 2, 1, 0, 2, 0, 5, 0, 14, 1], # d14
              [2, 0, 2, 11, 8, 1, 26, 1, 8, 1, 81, 2], # d15
              [0, 0, 0, 9, 5, 5, 13, 0, 4, 1, 47, 0], # d16
              [2, 0, 1, 9, 0, 0, 4, 0, 0, 0, 23, 7], # d17
              [1, 0, 1, 6, 0, 0, 1, 0, 0, 0, 16, 6], # d18
              [11, 0, 2, 199, 18, 53, 259, 0, 87, 4, 26, 0], # d19
              [2, 0, 0, 3, 0, 0, 1, 0, 0, 0, 12, 2], # d20
              ])

# comprobamos los totales por fila de la transparencia
print([i.sum() for i in F])

[24, 33, 8, 114, 127, 23, 66, 37, 19, 33, 28, 20, 27, 28, 143, 84, 46, 31, 659, 20]


Consulta original $q$

In [2]:
q = np.array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0])

Obtenemos la matriz de pesos tf-idf para que sea compatible con el código de prácticas anteriores

In [3]:
from collections import Counter
freqvector = {"d"+str(i+1): Counter({"t"+str(j+1): F[i][j] for j in range(len(F[i])) if F[i][j] != 0}) for i in range(len(F))}

Usamos código de P1

In [4]:
# Guardamos el vocabulario (el conjunto de todas las palabras que apaercen en los documentos de la colección).
vocabulary = set()
for word in freqvector.values(): vocabulary.update(word)

# Document frequency de cada palabra del vocabulario: nº de documentos que contienen la palabra.
docfreqs = {word:len([url for url in freqvector if word in freqvector[url]]) for word in vocabulary}

# Frecuencia total para cada palabra del vocabulario: nº total de apariciones en la colección.
wordfreqs = {word:sum([freqvector[url][word] for url in freqvector if word in freqvector[url]]) for word in vocabulary}


Incluid aquí vuestro código de P1 de la clase VSM (**completar lo que falta**):

In [5]:
class VSM:
  def __init__(self, freqvector, docfreqs):
    self.freqvector = freqvector
    self.docfreqs = docfreqs

  def search(self, q):
    # Calculamos los cosenos de todos los documentos.
    ranking = [(url, self.dotproduct(url, q) /  self.module(url)) for url in self.freqvector]
    # Eliminamos los documentos con coseno = 0.
    ranking = [(url, cos) for url, cos in ranking if cos > 0]
    # Ordenamos.
    ranking.sort(key=lambda x: x[1], reverse=True)
    return ranking
    
  def dotproduct(self, url, q):
    # Código aquí
    # ...
    product = 0
    for i in q:
      product += self.tf(i, url)*self.idf(i)

    return product

  def dotproduct_new(self, url, q, qw):
    # Código aquí
    # ...
    product = 0
    for term, weight in zip(q, qw):
      product += (self.tf(term, url)*self.idf(term))*weight

    return product

  def search_new(self, q, qw):
    # Calculamos los cosenos de todos los documentos.
    ranking = [(url, self.dotproduct_new(url, q, qw) /  self.module(url)) for url in self.freqvector]
    # Eliminamos los documentos con coseno = 0.
    ranking = [(url, cos) for url, cos in ranking if cos > 0]
    # Ordenamos.
    ranking.sort(key=lambda x: x[1], reverse=True)
    return ranking

  def module(self, url):
    # Código aquí
    # ...
    return np.linalg.norm(list(self.freqvector[url].values())) #freq

  def tf_idf_matrix(self, q):
    return np.array([[self.tf(i, url) * self.idf(i) for i in q] for url in self.freqvector])


  def tf(self, word, url):
    # Código aquí
    # ...
    if self.freqvector[url][word] > 0:
      return 1 + np.log2(self.freqvector[url][word])
    else:
      return 0

  def idf(self, word):
    return np.log2((len(self.freqvector)+1)/(self.docfreqs[word]+0.5))

Ahora ya podemos calcular la matriz de pesos tf-idf

In [6]:
vsm = VSM(freqvector, docfreqs)
def get_doc_vector(matrix, row):
    return [vsm.tf("t"+str(j+1), "d"+str(row+1))*vsm.idf("t"+str(j+1)) for j in range(len(matrix[row]))]
T = np.array([get_doc_vector(F, i) for i in range(len(F))])
print(T) # el resultado puede cambiar según la definición de TF e IDF

[[ 0.          1.9328858   1.49692247  0.36572811  0.63742992  1.6918777
   0.69584661  2.60970916  1.73751093  1.          0.12463272  1.91228976]
 [ 0.          1.9328858   2.2453837   0.36572811  1.27485984  0.
   0.          3.91456374  3.62264514  1.          0.15025364  0.        ]
 [ 0.74846123  0.          1.49692247  0.          0.          0.
   0.3479233   2.60970916  0.          1.          0.03476542  0.        ]
 [ 0.74846123  1.9328858   0.          0.18286406  2.65803496  1.6918777
   2.02051411  1.30485458  3.3076604   1.          0.23666054  2.54971968]
 [ 0.          0.          0.          0.18286406  2.42692195  3.38375541
   2.05514489  1.30485458  3.11445577  1.          0.24490131  2.99620092]
 [ 0.          0.          0.74846123  0.47269673  1.64773244  0.
   0.3479233   1.30485458  0.86875547  0.          0.11548822  2.54971968]
 [ 0.74846123  0.          1.49692247  0.36572811  1.91228976  0.
   0.          0.          0.          0.          0.23666054  0.6

Calculamos la consulta modificada $\vec{q}_m$ (**completar lo que falta**)

In [7]:
rel_click = np.array([1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1])
alpha = 1.0
beta = 0.8
gamma = 0.4

dr = np.array([T[i] for i in range(len(rel_click)) if rel_click[i] == 1])
dn = np.array([T[i] for i in range(len(rel_click)) if rel_click[i] == 0])

def get_rf_score(q0, dr, dn, alpha, beta, gamma): #stadard rocchio
    qm = alpha * q0
    for i in range(len(dr)):
      qm += beta * dr[i]
    for j in range(len(dn)):
      qm -= gamma * dn[j]
    return qm
    
def get_term_weight(q, term, matrix, dr, dn, alpha, beta, gamma):
    qm = alpha * q
    for i in range(len(dr)):
      qm += beta * dr[i]
    for j in range(len(dn)):
      qm -= gamma * dn[j]
    return qm[term]

In [8]:
get_rf_score(q, dr, dn, alpha, beta, gamma)

array([ 1.71643521,  7.73154322,  5.98768986,  3.44406727,  4.55832364,
       -2.06398831,  4.82286545,  9.39495299,  9.14813321,  2.4       ,
        2.27627416,  7.67687416])

In [9]:
get_term_weight(q, 0, T, dr, dn, alpha, beta, gamma)

1.7164352059307313

# 1.  Práctica

## 1.1 Cálculo de relevance feedback

Sobre alguna consulta que hayas usado en prácticas anteriores (para la que tengas calculada los valores de TF-IDF), aplicar relevance feedback (una iteración) según las siguientes opciones.

1. Sistema/ránking de partida
    1. Google (obtenido en el proceso de recogida de datos).
    2. **Modelo vectorial (VSM obtenido en la práctica 3 ej. 1).**
    3. PRP modelo BIR ó document likelihood modelo 1 ó 2.
2. Técnica RF a usar:
    1. **Para Google y VSM, expansión de consulta por el método Rocchio.**
    2. Para los modelos probabilísticos, uso del conjunto de documentos feedback en la reestimación de probabilidades, según corresponde a cada método.
3. Fuente de feedback:
    * i) Clicks como feedback de relevancia implícita positiva usando el log de clicks de la práctica 2.
    * **ii) Pseudo-relevance feedback (PRF): los documentos top n devueltos por el propio sistema. Nota: esta es la opción natural de BIR y document likelihood, pero tienen igualmente sentido el uso de clicks o juicios de relevancia (como prueba de concepto).**
    * iii) Oráculo: juicios de relevancia (de la práctica 2) como feedback explícito de relevancia positiva y negativa. Nota: esta opción no corresponde a un escenario real (donde el feedback lo daría un usuario final en lugar de un asesor) ni a una evaluación “justa” en laboratorio (pues le enseñamos al sistema las “soluciones” a la pregunta sobre la que va a ser evaluado), pero sirve como prueba de contraste con las otras opciones.

Quedan a criterio del estudiante:
 - Los parámetros alfa, beta y gamma en Rocchio.
 - La profundidad del cutoff top-n en PRF.
 - El número de términos en la expansión de consultas y el criterio para seleccionarlos.

Observación: en el apartado 1.1, la interfaz de Google no contempla la ponderación de términos de consulta, de modo que se añadirán a la consulta términos sin peso. Cabría considerar pesos enteros (p.e. multiplicando los pesos por una constante y redondeándolos), que se pueden plasmar como términos repetidos en la consulta. El estudiante puede experimentar si lo desea con esta idea.

###1. SISTEMA/RANKING DE PÉRDIDA 
Para ello, hacemos uso del modelo vectorial SVM previamente definido, y el conjunto de datos proveniente de la práctica 1.

In [10]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from collections import Counter
import re

# La colección: una pequeña lista de URLs web.
urls = ["https://en.wikipedia.org/wiki/Age_of_Enlightenment",
        "https://en.wikipedia.org/wiki/Rationalism",
        "https://en.wikipedia.org/wiki/Scientific_Revolution",
        "https://en.wikipedia.org/wiki/French_Revolution",
        "https://en.wikipedia.org/wiki/Winner%27s_curse",
        "https://en.wikipedia.org/wiki/Simpson%27s_paradox",
        "https://en.wikipedia.org/wiki/Friendship_paradox",
        "https://en.wikipedia.org/wiki/Condorcet_paradox",
        "https://en.wikipedia.org/wiki/Paradox_of_value",
        "https://en.wikipedia.org/wiki/Ship_of_Theseus"
       ]

relevance = [1, 0, 1, 0, 0, 0, 0, 0, 0, 2]

relevance_dict = {}

for i in range(len(urls)):
  relevance_dict[urls[i]] = relevance[i]
# Leemos los documentos y quitamos las marcas HTML.
texts = [BeautifulSoup(urlopen(url).read(), "lxml").text.lower() for url in urls]

# Una lista ad-hoc de stopwords.
stoplist = ["also", "could", "p", "pp", "th", "however", "one", "two", "many", "i", "de", "la", "me", "my", "myself", "the", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"]

In [30]:
# Nos abstraemos de los detalles reales de indexación, y utilizaremos un manejo muy simplificado del texto.

# Vector de frecuencias para todos los documentos de la colección, usando la subclase de diccionario collections.Counter.
# Para cada documento, separamos el texto en lista de palabras, y Counter genera un diccionario palabra:frecuencia.
# Se construye un diccionario url -> word -> count (se denomina un "índice forward").
freqvector = {url:Counter([word for word in re.findall(r"[^\W\d_]+|\d+", text) if word not in stoplist]) for url, text in zip(urls, texts)}
# Guardamos el vocabulario (el conjunto de todas las palabras que apaercen en los documentos de la colección).
vocabulary = set()
for word in freqvector.values(): vocabulary.update(word)

# Document frequency de cada palabra del vocabulario: nº de documentos que contienen la palabra.
docfreqs = {word:len([url for url in freqvector if word in freqvector[url]]) for word in vocabulary}

# Frecuencia total para cada palabra del vocabulario: nº total de apariciones en la colección.
wordfreqs = {word:sum([freqvector[url][word] for url in freqvector if word in freqvector[url]]) for word in vocabulary}

# Probamos tres consultas.
data_df = []
q = ['thought', 'experiment', 'identity']
print('\n------------------------------')
print('Query:', q)
print('\nModelo vectorial')
for url, score in VSM(freqvector, docfreqs).search(q):
  print(score, url)
  data_df.append((url, score, relevance_dict[url]))


------------------------------
Query: ['thought', 'experiment', 'identity']

Modelo vectorial
0.13984374069999364 https://en.wikipedia.org/wiki/Ship_of_Theseus
0.024608498606085397 https://en.wikipedia.org/wiki/Scientific_Revolution
0.01750852510522708 https://en.wikipedia.org/wiki/Friendship_paradox
0.015200399763869204 https://en.wikipedia.org/wiki/Rationalism
0.011377769622261988 https://en.wikipedia.org/wiki/Paradox_of_value
0.010586809838394183 https://en.wikipedia.org/wiki/Age_of_Enlightenment
0.008415728106016939 https://en.wikipedia.org/wiki/French_Revolution


Podemos ver que hay resultados que consideramos relevantes que no han puntuado especialmente alto en el ránking. 

In [12]:
base_df = pd.DataFrame(data_df, columns=["url", "score", "rel"])

### 2. TÉCNICA RF A USAR
En base a nuestro Pseudo Relevance Feedback (PRF), hacemos uso de la expansión de consulta por el método Rocchio, seleccionando términos en base a Robertson Selection Value

In [13]:
vsm = VSM(freqvector, docfreqs)
q0 = []
for word in vocabulary:
  if word in q:
    q0.append(1)
  else:
    q0.append(0)

q0 = np.array(q0)
T = vsm.tf_idf_matrix(vocabulary)

In [14]:
T

array([[1.28950662, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [2.57901323, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 2.13750352, 0.        , ..., 2.13750352, 2.87446912,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [15]:
alpha = 1.0
beta = 0.8
gamma = 0.4
dr = []
dn = []

for i in range(len(urls)):
  if relevance[i] > 0:
    dr.append(T[i])
  else:
    dn.append(T[i])

dr = np.array(dr)
dn = np.array(dn)

In [16]:
qm = get_rf_score(q0, dr, dn, alpha, beta, gamma)
qm

array([-1.03160529,  0.85500141, -1.14978765, ...,  0.        ,
        2.29957529, -1.14978765])

### Implementación Robertson Selection Value

In [17]:
def rsv_terms(T, dr_index, dr):
    R = dr.shape[0]
    rsv = np.zeros(len(vocabulary))
    D = T.shape[0]

    for i, w in enumerate(vocabulary):
      Dw = np.sum(T[:,i] > 0)
      rw = np.sum(T[dr_index, i] > 0)
      rsv[i] = rw*(np.log10((rw + 0.5)/(R - rw + 0.5)) + np.log10((D - Dw + rw - R + 0.5)/(Dw - rw + 0.5)))
    rsv_rank = rsv.argsort()[::-1]

    return rsv_rank, rsv[rsv_rank]

In [18]:
top_ranking_index = [i for i in range(len(urls)) if relevance[i] > 0]
rsv_rank, rsv = rsv_terms(T, top_ranking_index, dr)

### Nuevos términos a añadir en la query + ránking

In [19]:
np.array(list(vocabulary))[rsv_rank[:10].astype(int)]

array(['forth', 'research', 'edinburgh', 'referring', 'organizations',
       'scientists', 'formulation', 'facts', 'components', 'standing'],
      dtype='<U164')

In [20]:
extended_query = q
query_weights = []
for term in np.array(list(vocabulary))[rsv_rank[:10].astype(int)]:
  if term not in extended_query:
    extended_query.append(term)

for term, weight in zip(list(vocabulary), qm):
  if term in extended_query:
    query_weights.append(weight)

data_df = []
print('\n------------------------------')
print('Query:', extended_query)
print('\nModelo vectorial')
for url, score in VSM(freqvector, docfreqs).search_new(extended_query, query_weights):
  print(score, url)
  data_df.append((url, score, relevance_dict[url]))


------------------------------
Query: ['thought', 'experiment', 'identity', 'forth', 'research', 'edinburgh', 'referring', 'organizations', 'scientists', 'formulation', 'facts', 'components', 'standing']

Modelo vectorial
2.8047759539931816 https://en.wikipedia.org/wiki/Ship_of_Theseus
0.7482469613865828 https://en.wikipedia.org/wiki/Scientific_Revolution
0.4815423540455812 https://en.wikipedia.org/wiki/Age_of_Enlightenment
0.12860035553369706 https://en.wikipedia.org/wiki/Friendship_paradox
0.060269342948601076 https://en.wikipedia.org/wiki/Rationalism
0.04511267532477929 https://en.wikipedia.org/wiki/Paradox_of_value
0.0446396169116309 https://en.wikipedia.org/wiki/French_Revolution


In [21]:
rf_df = pd.DataFrame(data_df, columns=["url", "score", "rel"])

## 1.2 Comparar efectividad

Comparar la efectividad de los ránkings resultantes de aplicar relevance feedback en comparación con el ránking inicial, en términos de precisión y nDCG. 

Nota: para Google (apartado 2.1), el propio estudiante tendrá que añadir juicios de relevancia a los nuevos documentos que pueda devolver Google para la consulta expandida.

In [22]:
def precision(ranking_df):

  returned = len(ranking_df)
  intersect = len(ranking_df[ranking_df['rel'] > 0])

  return intersect/returned


def _nDCG(ranking_df, k):
    
    group = ranking_df
    ndcg_values = []
    
    topk_scores = group['rel'].values[:k]
    topk_positions = np.arange(1, k+1)
    topk_scores = [int(i) for i in topk_scores]
    topk_positions = [int(i) for i in topk_positions]
    # Compute DCG
    dcg = sum([(2**topk_scores[i] - 1)/np.log2(topk_positions[i] + 1) for i in range(len(topk_scores))])

    # Compute IDCG
    ideal_scores = np.sort(group['rel'].values)[::-1][:k]
    ideal_positions = np.arange(1, k+1)
    ideal_scores = [int(i) for i in ideal_scores]
    ideal_positions = [int(i) for i in ideal_positions]
    idcg = sum([(2**ideal_scores[i] - 1) / np.log2(ideal_positions[i] + 1) for i in range(len(ideal_scores))])

    # Compute nDCG
    if idcg > 0:
        return dcg / idcg
    else:
        return 0

### Comparativa resultados

In [23]:
print("Base Query vs Relevance Feedback")
print("---PRECISION----")
print(f"Base: {precision(base_df)} vs RF: {precision(rf_df)}")
print("---nDCG@5----")
print(f"Base: {_nDCG(base_df,5)} vs RF: {_nDCG(rf_df,5)}")

Base Query vs Relevance Feedback
---PRECISION----
Base: 0.42857142857142855 vs RF: 0.42857142857142855
---nDCG@5----
Base: 0.878961873034099 vs RF: 1.0


## 1.3 Informe

Escribir un informe con los resultados y detalles de los métodos y fórmulas aplicados, así como las diferencias observadas entre los resultados de las diferentes opciones y modelos.

**Respuesta:** Para el funcionamiento correcto del método de expansión de queries mediante Relevance Feedback hemos tenido que realizar algunos cambios en la implementación de VSM de la práctica 1 para tener en cuenta los pesos de los términos de la query (métodos dotproduct_new y search_new), así como para generar la matriz tf-idf completa.

Hemos optado por utilizar el ránking de la práctica 1 para una de las queries, sumado a un pseudo-relevance feedback (PRF) que hemos determinado en base a criterio personal. Esto nos ha permitido determinar que la query base no coloca todos los documentos relevantes en los primeros puestos del ránking, lo cual ofrece la oportunidad de probar el funcionamiento de este método.

Como método de expansión de query utilizamos Rocchio, que aparece implementada en la función *get_rf_score*, donde revalorizamos los términos del vocabulario en función de la relevancia de los documentos, beneficiando a los términos de documentos relevantes en un factor $\beta$ = 0.8 y perjudicando a los términos de documentos no relevantes en un factor $\gamma$ = 0.4.

Con este método pasamos de tener una query donde la inmensa mayoría de los datos son 0, a una query muy grande donde tenemos que la mayoría de valores son distintos de 0. Para solventar este problema, implementamos la función *rsv_terms*, donde aplicamos Robertson Selection Value (RSV), para seleccionar los 10 mejores términos que mejor se ajusten a la query para expandir la base.

Una vez obtenidos los términos y los pesos asociados a cada uno, extendemos la query y comprobamos los resultados entre la base y la nueva query en base a precisión y nDCG@5. La precisión se ha mantenido igual, porque ambas devuelven los mismos documentos relevantes y el mismo número de documentos (0.4285), pero el valor de nDCG@5 ha mejorado (0.8790 vs 1.0). Esto se refleja en que ahora tiene todos los documentos relevantes en las primeras posiciones del ránking, mostrando que la query extendida con relevance feedback funciona correctamente.


## Opcional A

Probar a realizar varias iteraciones de relevance feedback en aquellos métodos donde sea viable y/o tenga más sentido (p.e. es más difícil con buscadores comerciales, ya que entrarían nuevos documentos de los que habría que extraer el vector de palabras, y -salvo para PRF- obtener nuevos clicks y/o juicios de relevancia).

In [ ]:
# TODO

## Opcional B

Probar valores diferentes de alfa, beta y gamma en Rocchio, diferentes cutoff en PRF, y/o diferente nº de términos de expansión.

Vamos a aplicar la estrategia para Rocchio propuesta por Ide. Esta consiste en poner los parámetros $\alpha = \beta = \gamma = 1$.

Además, vamos a aplicar un cutoff con solo los 3 mejores términos de expansión.

In [25]:
qm = get_rf_score(q0, dr, dn, alpha=1.0, beta=1.0, gamma=1.0)
qm

array([-3.86851985,  0.        , -2.87446912, ..., -2.13750352,
        2.87446912, -2.87446912])

In [26]:
top_ranking_index = [i for i in range(len(urls)) if relevance[i] > 0]
rsv_rank, rsv = rsv_terms(T, top_ranking_index, dr)

In [28]:
np.array(list(vocabulary))[rsv_rank[:3].astype(int)]

array(['forth', 'research', 'edinburgh'], dtype='<U164')

In [31]:
extended_query = q
query_weights = []
for term in np.array(list(vocabulary))[rsv_rank[:3].astype(int)]:
  if term not in extended_query:
    extended_query.append(term)

for term, weight in zip(list(vocabulary), qm):
  if term in extended_query:
    query_weights.append(weight)

data_df = []
print('\n------------------------------')
print('Query:', extended_query)
print('\nModelo vectorial')
for url, score in VSM(freqvector, docfreqs).search_new(extended_query, query_weights):
  print(score, url)
  data_df.append((url, score, relevance_dict[url]))


------------------------------
Query: ['thought', 'experiment', 'identity', 'forth', 'research', 'edinburgh']

Modelo vectorial
1.4248219831779598 https://en.wikipedia.org/wiki/Ship_of_Theseus
0.36846720410693645 https://en.wikipedia.org/wiki/Scientific_Revolution
0.25526135104939246 https://en.wikipedia.org/wiki/Age_of_Enlightenment
0.14262123530885204 https://en.wikipedia.org/wiki/Friendship_paradox
0.07533667868575135 https://en.wikipedia.org/wiki/Rationalism
0.06743246822341542 https://en.wikipedia.org/wiki/French_Revolution
0.05639084415597411 https://en.wikipedia.org/wiki/Paradox_of_value


In [32]:
rf_df = pd.DataFrame(data_df, columns=["url", "score", "rel"])

In [33]:
print("Base Query vs Relevance Feedback")
print("---PRECISION----")
print(f"Base: {precision(base_df)} vs RF: {precision(rf_df)}")
print("---nDCG@5----")
print(f"Base: {_nDCG(base_df,5)} vs RF: {_nDCG(rf_df,5)}")

Base Query vs Relevance Feedback
---PRECISION----
Base: 0.42857142857142855 vs RF: 0.42857142857142855
---nDCG@5----
Base: 0.878961873034099 vs RF: 1.0


Podemos comprobar que a nivel de métricas no cambia nada los resultados con respecto a la expansión de query con más términos. Sin embargo, si evaluamos los scores, es cierto que los documentos relevantes tienen un valor de score claramente menor que la expansión originalmente propuesta, aunque el ranking sufre una pequeña alteración en tanto que los dos últimos documentos del ránking intercambian lugares.

Estos cambios realmente no aportan nada nuevo a nivel de resultados, pero tenemos cierta seguridad al afirmar que esto se debe al bajo número de documentos extraídos y la nula relevancia de la mayoría de ellos. Presuponemos que estos cambios, en una query realizada de verdad en un buscador sería mucho más interesante de cara a ver cómo se comportan los distintos parámetros en Rocchio, además de ver como los diferentes cutoffs en términos de expansión alterarían el ránking de los documentos.

## Opcional C

Probar con diferentes consultas y comparar si los resultados que se obtienen son consistentes con los obtenidos usando la consulta de los primeros apartados.

In [ ]:
# TODO